In [16]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pathlib import Path
import sys
import os
current_dir = os.getcwd()  # Lấy thư mục hiện tại
root_dir = os.path.abspath(os.path.join(current_dir, '..'))  # Lên 2 cấp thư mục
sys.path.insert(0, root_dir)
from config import RAW_DATA_DIR, PROCESSED_DATA_DIR, CACHE_DIR, MODEL_PARAMS, MODELS_DIR

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("F1DataProcessing") \
    .config("spark.sql.legacy.parquet.nanosAsLong", "true") \
    .getOrCreate()

def read_parquet_with_columns(path, cols):
    return spark.read.parquet(path).select(cols)


In [ ]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("F1DataProcessing").getOrCreate()

years = [2021, 2022, 2023]
processed_dfs = []

def read_parquet_with_columns(path, cols):
    return spark.read.parquet(path).select(cols)

for year in years:
    # Đọc danh sách các file results của năm đó (ví dụ: lấy tất cả file results trong thư mục năm)
    results_files = [f"{RAW_DATA_DIR}/{year}/monaco_grand_prix_r_results.parquet"]  # Thay bằng cách lấy tất cả file results của năm

    for results_file in results_files:
        # Lấy tên GP từ tên file (ví dụ: monaco_grand_prix)
        gp_name = results_file.split('/')[-1].split('_results')[0]

        # Đọc results
        results_cols = ["drivernumber", "driverid", "teamname", "fullname", "position", "classifiedposition", "gridposition", "status", "points"]
        results = read_parquet_with_columns(results_file, results_cols)
        results = results.withColumn("year", F.lit(year))
        results = results.withColumn("gp_name", F.lit(gp_name))

        # Đọc weather
        weather_file = f"{RAW_DATA_DIR}/{year}/{gp_name}_weather.parquet"
        weather = spark.read.parquet(weather_file)
        weather_avg = weather.agg(
            *[F.avg(col).alias(f"avg_{col}") for col in weather.columns if col not in ["drivernumber", "year", "gp_name"]]
        )

        # Đọc laps
        laps_file = f"{RAW_DATA_DIR}/{year}/{gp_name}_laps.parquet"
        laps_cols = ["drivernumber", "lapnumber", "stint", "pitintime", "pitouttime", "time", "speed", "compound", "tyrelife", "trackstatus", "position"]
        laps = read_parquet_with_columns(laps_file, laps_cols)
        # Tính toán các chỉ số theo driver (min, max, mean)
        stats = laps.groupBy("drivernumber").agg(
            F.count("lapnumber").alias("lap_count"),
            F.min("time").alias("min_time_ms"),
            F.max("time").alias("max_time_ms"),
            F.mean("time").alias("avg_time_ms"),
            F.min("speed").alias("min_speed"),
            F.max("speed").alias("max_speed"),
            F.mean("speed").alias("avg_speed"),
            F.countDistinct("compound").alias("compound_count"),
        )
        # Ghép vào results
        results = results.join(stats, on="drivernumber", how="left")

        processed_dfs.append(results)

# Gộp tất cả các results đã xử lý
final_results = processed_dfs[0]
for df in processed_dfs[1:]:
    final_results = final_results.union(df)

# Lưu kết quả
final_results.write.mode("overwrite").parquet(f"{PROCESSED_DATA_DIR}/f1_processed_results.parquet")


AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "avg(Rainfall)" due to data type mismatch: Parameter 1 requires the "NUMERIC" or "ANSI INTERVAL" type, however "Rainfall" has the type "BOOLEAN".;
'Aggregate [avg(Time#899L) AS avg_Time#943, avg(AirTemp#900) AS avg_AirTemp#945, avg(Humidity#901) AS avg_Humidity#947, avg(Pressure#902) AS avg_Pressure#949, avg(Rainfall#903) AS avg_Rainfall#951, avg(TrackTemp#904) AS avg_TrackTemp#953, avg(WindDirection#905L) AS avg_WindDirection#955, avg(WindSpeed#906) AS avg_WindSpeed#957]
+- Filter rainfall#903: boolean
   +- Relation [Time#899L,AirTemp#900,Humidity#901,Pressure#902,Rainfall#903,TrackTemp#904,WindDirection#905L,WindSpeed#906] parquet
